In [26]:
from __future__ import print_function
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.action_chains import ActionChains
import os
import logging
from parsel import Selector
from pprint import *
import time
from peewee import *
from playhouse.sqlite_ext import SqliteExtDatabase
import datetime
import re
import copy
import pandas as pd
from IPython.display import display, HTML
import psycopg2
import json
from w3lib.url import url_query_parameter
import w3lib.url
import requests

In [22]:
class BaseModel(Model):
    class Meta:
        db = SqliteDatabase("kvk_final.db")
        db_connected = db.connect()
        database = db
    
class kvkItemModel(BaseModel):
    search_string = CharField()
    eight_digit_kvk = CharField(null=True, unique=True)
    twelve_digit_kvk = CharField(null=True)
    vesting_nr = CharField(null=True)
    status_of_company = CharField(null=True)
    headquarters = BooleanField(null=True)    
    main_company_name = CharField(null=True)
    bestaande_handelsnamen = CharField(null=True)
    statutaire_naam = CharField(null=True)
    second_page_link = CharField(null=True)
    street = CharField(null=True)
    housenumber = CharField(null=True)
    housenumber_extension = CharField(null=True)
    postcode = CharField(null=True)
    city = CharField(null=True)
    extra_infomation = CharField(null=True)
    second_page_company_name = CharField(null=True)
    website = CharField(null=True)
    

    
    

db = SqliteDatabase("kvk_final.db")
db.connect()
try:
    db.drop_tables([kvkItemModel])
except:
    pass
db.create_tables([kvkItemModel])
print("="*30)
print("3rd sqlite database successfully inited!")
print("="*30)

3rd sqlite database successfully inited!


In [23]:
db = SqliteDatabase("temp_kvk_1.db")
db.connect()

class BaseModel(Model):
    class Meta:
        database = db

class kvkFirstStageModel(BaseModel):
    url = CharField(unique=True)
    post_code = CharField(unique=False)

In [24]:
first_stage_results = kvkFirstStageModel.select()
first_stage_urls = []
for result in first_stage_results:
    first_stage_urls.append(copy.deepcopy(result.url))

url = first_stage_urls[0]

In [25]:
url

'https://zoeken.kvk.nl/JsonSearch.ashx?q=1000AA&prefproduct=&prefpayment=&start=0'

In [33]:
response = requests.get(url)
json_obj = json.loads(response.text)

response_url = response.url
for company_obj in json_obj['entries']:
    item = kvkItemModel()
    
    print("="*30)
    print("Parsing Company information -- Stage 2")
    search_string = url_query_parameter(response_url, 'q')
    item.search_string = search_string
    try:
        item.eight_digit_kvk = str(company_obj['dossiernummer']).strip()
    except Exception as e:
        print("8 Digit Corrupted. ")
        
    try:
        twelve_digit_kvk = url_query_parameter(str(company_obj['href']), 'kvknummer')
        item.twelve_digit_kvk = twelve_digit_kvk
        if len(item.twelve_digit_kvk) <= 11:
            item.twelve_digit_kvk = str(item.twelve_digit_kvk) + "0000"
    except Exception as e:
        print("12 Digit Corrupted. ")
    
    try: 
        item.vesting_nr = str(company_obj['vestigingsnummer'])
    except Exception as e:
        print("Vesting NR Corrupted")
    
    try:
        item.status_of_company = str(company_obj['status']).strip()
    except Exception as e:
        print("This company do not have a status. ")
    
    try:
        if (company_obj['type']).strip().startswith("H") is True:
            item.headquarters = True
        else:
            item.headquarters = False
    except Exception as e:
        print("This company is not a headquater. ")
    
    try:
        item.main_company_name = str(company_obj['handelsnaam'])
        print("********This company name is `%s`********" % str(item.main_company_name))
    except Exception as e:
        print("Main company name Corrupted. ")
    
    try:
        for meerinfo in company_obj['meerinfo']:
            if meerinfo['titel'].startswith("Be") is True:
                item.bestaande_handelsnamen = str(meerinfo['tekst'])
            else:
                pass
    except Exception as e:
        print("This company do not have a Bestaande handelsnamen. ")
    if item.bestaande_handelsnamen is None:
        print("This company do not have a Bestaande handelsnamen. ")

    try:    
        for meerinfo in company_obj['meerinfo']:
            if meerinfo['titel'].startswith("Stat") is True:
                item.statutaire_naam = str(meerinfo['tekst'])
            else:
                pass
    except Exception as e:
        print("This company do not have a Statutaire naam. ")
    if item.statutaire_naam is None:
        print("This company do not have a Statutaire naam. ")
    
    try:
        item.second_page_link = "https://www.kvk.nl/"+str(company_obj['href'])
    except Exception as e:
        print("Second Page Link Corrupted. ")
    
    try:
        item.street = str(company_obj['straat'])
    except Exception as e:
        print("This company's location do not have a street. ")
    
    try:
        item.housenumber = str(company_obj['huisnummer'])
    except Exception as e:
        print("This company's location do not have a house number. ")
        
    try:
        item.housenumber_extension = str(company_obj['huisnummertoevoeging'])
    except Exception as e:
        print("This company's house number do not have an extension. ")
        
    try:
        item.postcode = str(company_obj['postcode'])
    except Exception as e:
        print("This company do not have a postcode. ")
        
    try:
        item.city = str(company_obj['plaats'])
    except Exception as e:
        print("This company's location do not have a city string. ")
    
    try:
        item.extra_infomation = str(company_obj['snippet'])
    except Exception as e:
        print("This company do not include extra information. ")
    
    try:
        item.second_page_company_name = str(company_obj['handelsnaam'])
    except Exception as e:
        print("Second page company name corrupted. ")
    
    try:
        website = re.search(r'(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9]\.[^\s]{2,})',
                           company_obj['snippet'],
                           flags=re.IGNORECASE)
        if website is not None:
            item.website = str(website)
        else:
            print("This company do not have a website information. ")
    except:
        print("This company do not have a website information. ")

    try:
        item.save()
        print("This is a valid record. ")
    except Exception as e:
        # This means we have inserted a duplicate record.
        print("This is a duplicate record")
    
    print("+"*30)

Parsing Company information -- Stage 2
This company do not have a status. 
********This company name is `First Holding B.V.`********
This is a duplicate record
++++++++++++++++++++++++++++++
Parsing Company information -- Stage 2
This company do not have a status. 
********This company name is `Xytrium Life B.V.`********
This is a duplicate record
++++++++++++++++++++++++++++++
Parsing Company information -- Stage 2
This company do not have a status. 
********This company name is `First Consulting B.V.`********
This is a duplicate record
++++++++++++++++++++++++++++++
Parsing Company information -- Stage 2
This company do not have a status. 
********This company name is `Village Bagels`********
This company do not have a Statutaire naam. 
This is a duplicate record
++++++++++++++++++++++++++++++
Parsing Company information -- Stage 2
This company do not have a status. 
********This company name is `Roeminck Insurance N.V.`********
This company do not have a website information. 
This i